## We will be working with database chinook.db
You can download it here: https://drive.google.com/file/d/0Bz9_0VdXvv9bWUtqM0NBYzhKZ3c/view?usp=sharing

The	Chinook	Database is	about an imaginary video and music store. Each track is	stored using one of	the digital formats and	has	a genre. The store has also	some playlists,	where a single track can be	part of several	playlists. Orders are recorded	for	customers, but are called invoices. Every customer is assigned a support employee, and Employees report to other employees.

### How to connect Python to database
In the begining of this exercise we are going to show we can work with the databases inside the Python. We will connect this notebook to the `chinook.db` file we have downloaded earlier.

<img src="chinook.jpeg" alt="Drawing" style="width: 600px;"/>

In [1]:
import sqlite3
conn = sqlite3.connect("chinook.db") # the chinook file has to be in the same directory as our notebook

In [2]:
cur = conn.cursor()
cur.execute("select * from artists limit 5;")
results = cur.fetchall()
print(results)

[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


- each row is represented as 1 item in the list
- each row consists of tuple with number of items equal to number of columns in the table `artists`

In [2]:
# Before we move on, it’s good practice to close Connection objects and Cursor objects that are open. 
# This prevents the SQLite database from being locked.
cur.close()
conn.close()

Now, we can solve simple challenges below. Feel free to use **SQLWorkbench**, the tool we connected to `chinook.db` database during the tutorial in the morning, for building the queries as well.

In [2]:
import sqlite3
conn = sqlite3.connect("chinook.db")
cur = conn.cursor()

### Which	artists	did	not	make	any	albums	at	all?	Include	their	names	in your	answer.

In [3]:
cur.execute('''SELECT artists.name FROM artists 
                   WHERE artists.name NOT IN 
                       (SELECT artists.name FROM artists 
                           JOIN albums ON artists.artistid == albums.artistid);''')
            
results = cur.fetchall()
print(results)

[('Milton Nascimento & Bebeto',), ('Azymuth',), ('João Gilberto',), ('Bebel Gilberto',), ('Jorge Vercilo',), ('Baby Consuelo',), ('Ney Matogrosso',), ('Luiz Melodia',), ('Nando Reis',), ('Pedro Luís & A Parede',), ('Banda Black Rio',), ('Fernanda Porto',), ('Os Cariocas',), ('A Cor Do Som',), ('Kid Abelha',), ('Sandra De Sá',), ('Hermeto Pascoal',), ('Barão Vermelho',), ('Edson, DJ Marky & DJ Patife Featuring Fernanda Porto',), ('Santana Feat. Dave Matthews',), ('Santana Feat. Everlast',), ('Santana Feat. Rob Thomas',), ('Santana Feat. Lauryn Hill & Cee-Lo',), ('Santana Feat. The Project G&B',), ('Santana Feat. Maná',), ('Santana Feat. Eagle-Eye Cherry',), ('Santana Feat. Eric Clapton',), ('Vinícius De Moraes & Baden Powell',), ('Vinícius E Qurteto Em Cy',), ('Vinícius E Odette Lara',), ('Vinicius, Toquinho & Quarteto Em Cy',), ('Motörhead & Girlschool',), ('Peter Tosh',), ('R.E.M. Feat. KRS-One',), ('Simply Red',), ('Whitesnake',), ('Christina Aguilera featuring BigElf',), ("Aerosmith

I will do the queries in SQLWorkbench but I will write them here in text.


### Which	artists	recorded	any	tracks	of	the	Latin	genre?

SELECT artists.name FROM artists
    JOIN albums 
    ON artists.artistid == albums.artistid
    WHERE albumid IN (SELECT albumid FROM tracks
                         WHERE genreid IN (SELECT genreid FROM genres
                                               WHERE name = "Latin"))
    GROUP BY artists.name;

In [4]:
cur.execute('''SELECT artists.name FROM artists
                  JOIN albums 
                  ON artists.artistid == albums.artistid
                  WHERE albumid IN (SELECT albumid FROM tracks
                                       WHERE genreid IN (SELECT genreid FROM genres 
                                                            WHERE name = "Latin"))
                  GROUP BY artists.name;''')
            
results = cur.fetchall()
print(results)

[('Antônio Carlos Jobim',), ('Caetano Veloso',), ('Chico Buarque',), ('Chico Science & Nação Zumbi',), ('Cláudio Zoli',), ('Cássia Eller',), ('Djavan',), ('Ed Motta',), ('Elis Regina',), ('Eric Clapton',), ('Falamansa',), ('Funk Como Le Gusta',), ('Gilberto Gil',), ('Gonzaguinha',), ('Jorge Ben',), ('Jota Quest',), ('Legião Urbana',), ('Luciana Souza/Romero Lubambo',), ('Lulu Santos',), ('Marcos Valle',), ('Marisa Monte',), ('Milton Nascimento',), ('Olodum',), ('Os Paralamas Do Sucesso',), ('Tim Maia',), ('Various Artists',), ('Vinícius De Moraes',), ('Zeca Pagodinho',)]


### Which	video track	has	the	longest	length?

SELECT tracks.name, MAX(tracks.milliseconds) FROM tracks
    JOIN media_types
    ON tracks.mediatypeid = media_types.mediatypeid
    WHERE tracks.mediatypeid IN (SELECT mediatypeid FROM media_types WHERE name LIKE "%video%");

In [5]:
cur.execute('''SELECT tracks.name, MAX(tracks.milliseconds) FROM tracks
                   JOIN media_types
                   ON tracks.mediatypeid = media_types.mediatypeid
                   WHERE tracks.mediatypeid IN 
                       (SELECT mediatypeid FROM media_types 
                           WHERE name LIKE "%video%");''')
            
results = cur.fetchall()
print(results)

[('Occupation / Precipice', 5286953)]


### Find	the	names	of	customers	who	live	in	the	same	city	as	the	top	employee	(The	one	not	managed	by	anyone).

In [ ]:
SELECT firstname, lastname FROM customers
    WHERE city IN (SELECT city FROM employees
                       WHERE reportsto IS NULL)  

In [6]:
cur.execute('''SELECT firstname, lastname FROM customers
                  WHERE city IN (SELECT city FROM employees
                                    WHERE reportsto IS NULL)''')
            
results = cur.fetchall()
print(results)

[('Mark', 'Philips')]


### Find	the	managers of	employees	supporting	Brazilian	customers.

In [ ]:
SELECT employees.lastname, employees.firstname FROM employees
    JOIN employees a
    ON employees.employeeid = a.reportsto
    WHERE employees.title LIKE "%Manager%" AND a.reportsto IN (SELECT reportsto FROM employees
                                                                  JOIN customers
                                                                  ON employees.employeeid = customers.supportrepid
                                                                  WHERE customers.country = "Brazil"
                                                                  GROUP BY employees.lastname)
    GROUP BY employees.lastname;

In [7]:
cur.execute('''SELECT employees.lastname, employees.firstname FROM employees
                  JOIN employees a
                  ON employees.employeeid = a.reportsto
                  WHERE employees.title LIKE "%Manager%" 
                  AND a.reportsto IN (SELECT reportsto FROM employees
                                          JOIN customers
                                          ON employees.employeeid = customers.supportrepid
                                          WHERE customers.country = "Brazil"
                                          GROUP BY employees.lastname)
                  GROUP BY employees.lastname;''')
            
results = cur.fetchall()
print(results)

[('Edwards', 'Nancy')]


### Which	playlists	have	no	Latin	tracks?

In [ ]:
SELECT playlists.playlistid, playlists.name FROM playlists
    WHERE playlists.playlistid NOT IN (SELECT playlists.playlistid FROM playlists
                                          JOIN playlist_track
                                          ON playlists.playlistid = playlist_track.playlistid
                                          JOIN tracks
                                          ON playlist_track.trackid = tracks.trackid
                                          WHERE genreid IN (SELECT genreid FROM genres
                                                               WHERE name = "Latin"));

In [8]:
cur.execute('''SELECT playlists.playlistid, playlists.name FROM playlists
                  WHERE playlists.playlistid NOT IN 
                      (SELECT playlists.playlistid FROM playlists
                          JOIN playlist_track
                          ON playlists.playlistid = playlist_track.playlistid
                          JOIN tracks
                          ON playlist_track.trackid = tracks.trackid
                          WHERE genreid IN (SELECT genreid FROM genres
                                               WHERE name = "Latin"));''')
            
results = cur.fetchall()
print(results)

[(2, 'Movies'), (3, 'TV Shows'), (4, 'Audiobooks'), (6, 'Audiobooks'), (7, 'Movies'), (9, 'Music Videos'), (10, 'TV Shows'), (12, 'Classical'), (13, 'Classical 101 - Deep Cuts'), (14, 'Classical 101 - Next Steps'), (15, 'Classical 101 - The Basics'), (16, 'Grunge'), (17, 'Heavy Metal Classic'), (18, 'On-The-Go 1')]


In [9]:
cur.close()
conn.close()